In [47]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.image as mpimg
import optuna
from itertools import cycle
import seaborn as sns
import statsmodels.api as sm 
from scipy.interpolate import interp1d
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from sklearn.metrics import mean_squared_error
import time

pd.set_option('display.max_columns', 50)
plt.style.use('bmh')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [48]:
%%time
oj = os.path.join
INPUT_DIR = '../input/m5-forecasting-accuracy/'
train = pd.read_csv(oj(INPUT_DIR, 'trainset.csv'))
price = pd.read_csv(oj(INPUT_DIR, 'sell_prices.csv'))
calendar = pd.read_csv(oj(INPUT_DIR, 'calendar.csv'))

sample_submit = pd.read_csv(oj(INPUT_DIR, 'submit.csv'))

CPU times: total: 1.77 s
Wall time: 5.64 s


In [49]:
holidays = calendar[['date', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']]
holidays = holidays.rename(columns={'date': 'ds'})
holidays['lower_window'] = 0
holidays['upper_window'] = 1
holidays1 = holidays.dropna(subset = ['event_name_1'])
holidays2 = holidays.dropna(subset = ['event_name_2'])
holidays1 = holidays1[['ds', 'lower_window', 'upper_window', 'event_name_1']]
holidays1 = holidays1.rename(columns={'event_name_1': 'holiday'})
holidays2 = holidays2[['ds', 'lower_window', 'upper_window', 'event_name_2']]
holidays2 = holidays2.rename(columns={'event_name_2': 'holiday'})
holidays = pd.concat((holidays1, holidays2))
holidays = holidays.reset_index(drop=True)
print(holidays.head())


           ds  lower_window  upper_window        holiday
0  2011-02-06             0             1      SuperBowl
1  2011-02-14             0             1  ValentinesDay
2  2011-02-21             0             1  PresidentsDay
3  2011-03-09             0             1      LentStart
4  2011-03-16             0             1      LentWeek2


In [53]:
def objective(trial):
    params = {'changepoint_prior_scale' : 
                 trial.suggest_uniform('changepoint_prior_scale',
                                       0.1,0.5
                                      ),
              'seasonality_prior_scale' : 
                 trial.suggest_uniform('seasonality_prior_scale',
                                       0.1,10
                                      ),
              'seasonality_mode' : 
                 trial.suggest_categorical('seasonality_mode',
                                           ['additive', 'multiplicative']
                                          ),
              'changepoint_range' : 
                  trial.suggest_discrete_uniform('changepoint_range', 
                                                 0.8, 1, 
                                                 0.1),
             }
    
    model = Prophet(holidays=holidays,
    **params)
    model.fit(train_opt)

    
    forecast =  model.predict(val_opt)
    
    val_rmse = np.sqrt(mean_squared_error(val_opt.y, forecast.yhat))
    return val_rmse



In [54]:
submit_df = pd.DataFrame()
start1 = time.time()
for i in range(10):
    start = time.time()
    df = pd.DataFrame(calendar[['date', 'd', 'wm_yr_wk', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']])
    end = time.time()
    print("df read", end-start)
    
    df_train =  df.loc[0:1926]
    df_test = df.loc[1927:]

    #学習用df編集，sell_price追加
    start = time.time()
    df_train['y'] = train.loc[i,'d_1':].values
    df_train = pd.merge(df_train, price[(price['store_id'] == train.loc[i,'store_id']) \
                & (price['item_id'] == train.loc[0,'item_id'])]\
                [['wm_yr_wk', 'sell_price']], how='left', on=['wm_yr_wk'])
    end = time.time()
    print("slice", end-start)
    
    df_train = df_train.rename(columns={'date': 'ds'})
    df_train = df_train.fillna({"sell_price":0})
    df_train[0] = False

    # # change point作成
    # index_0 = df_train['y'] == 0
    # index_change = index_0 != index_0.shift(1)

    # change_point = df_train[index_change]["ds"]
    # change_point = change_point.reset_index(drop=True)

    # # optuna
    train_opt = df_train[-200:-100]
    train_opt = train_opt[["ds", "y"]]

    val_opt = df_train[-100:]
    val_opt = val_opt[["ds", "y"]]


    # 最適化
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=5)


    #model定義   **study.best_params,\
    start = time.time()
    model = Prophet(weekly_seasonality=True, yearly_seasonality=True, holidays=holidays, **study.best_params, n_changepoints=15)
    end = time.time()
    print("model initialized", end-start)
    # model = Prophet(weekly_seasonality=True, yearly_seasonality=True)

    #周期生追加
    model.add_seasonality(name="monthly", period=30.5, fourier_order=14)
    model.add_seasonality(name="yearly", period=365.25, fourier_order=20)

    """"""
    #説明変数sell_price追加
    model.add_regressor('sell_price')

    model.fit(df_train)
    future = model.make_future_dataframe(periods=14)
    future["wm_yr_wk"] = calendar[:len(future)]["wm_yr_wk"]

    # futureにsell_price追加
    future = pd.merge(future, price[(price['store_id'] == train.loc[i,'store_id'])\
                    & (price['item_id'] == train.loc[0,'item_id'])] \
                    [['wm_yr_wk',"sell_price"]], how='left', on=['wm_yr_wk'])
    future = future.fillna({"sell_price":0})



    # 学習
    start = time.time()
    forecast = model.predict(future)
    end = time.time()
    print("predict", end-start)
    
    
    # fig = model.plot(forecast)
    # a = add_changepoints_to_plot(fig.gca(), model, forecast)

    # plt.show()
    """
    model.plot_components(forecast)
    plt.show()
    """
    #df_train.to_csv('df_train' + str(i) + '.csv', index=False)
    yhat_tenchi = forecast[['yhat']].tail(14).T.round(0)
    submit_df = pd.concat([submit_df, yhat_tenchi.reset_index(drop=True)], ignore_index=True)

end1 = time.time()
print("処理時間=", end1-start1)
for i, row in enumerate(submit_df.itertuples(index=False), start=0):
    # start=2 はsample_submitの2行目から始まるため
    # 7列目から始めて、行の各要素を置換する
    for j, value in enumerate(row, start=6):
        sample_submit.iat[i, j] = value

sample_submit.to_csv('submit_edited.csv', index=False)


df read 0.0010001659393310547


C:\Users\fuyuki\AppData\Local\Temp\ipykernel_53100\3653885342.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['y'] = train.loc[i,'d_1':].values
[I 2023-12-11 04:19:37,059] A new study created in memory with name: no-name-46529446-a655-4dd4-8005-901726500148


slice 0.5823326110839844


C:\Users\fuyuki\AppData\Local\Temp\ipykernel_53100\465002941.py:3: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  trial.suggest_uniform('changepoint_prior_scale',
C:\Users\fuyuki\AppData\Local\Temp\ipykernel_53100\465002941.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  trial.suggest_uniform('seasonality_prior_scale',
C:\Users\fuyuki\AppData\Local\Temp\ipykernel_53100\465002941.py:15: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  trial.suggest_discrete_uniform('changepoint_range',
04:19:37 - cmdstanpy - INFO - Chain [1] start processing


model initialized 0.1004798412322998


...

slice 0.6033506393432617


C:\Users\fuyuki\AppData\Local\Temp\ipykernel_53100\465002941.py:3: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  trial.suggest_uniform('changepoint_prior_scale',
C:\Users\fuyuki\AppData\Local\Temp\ipykernel_53100\465002941.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  trial.suggest_uniform('seasonality_prior_scale',
C:\Users\fuyuki\AppData\Local\Temp\ipykernel_53100\465002941.py:15: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  trial.suggest_discrete_uniform('changepoint_range',
04:20:36 - cmdstanpy - INFO - Chain [1] start processing


model initialized 0.10754656791687012


04:20:41 - cmdstanpy - INFO - Chain [1] start processing
04:20:41 - cmdstanpy - INFO - Chain [1] done processing


predict 0.4442787170410156
処理時間= 66.33067631721497
